In [ ]:
!pip install python-dotenv

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving eda 2_1.pdf to eda 2_1.pdf


In [ ]:
!pip install -qU langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 35.0 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate

In [ ]:
z# Load environment variables
# load_dotenv()
# GROQ_API_KEY ='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'


In [ ]:
# Initialize the chat model

model = init_chat_model("gemma2-9b-it", model_provider="groq", api_key=GROQ_API_KEY)


In [ ]:
# Load PDF Document
loader = PyPDFLoader('/content/eda 2_1.pdf')
docs = loader.load()

In [ ]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [ ]:
# Initialize embeddings and vector store
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")#sentences to numbers
vectorstore = FAISS.from_documents(splits, embedding_model)# converts into vectors
retriever = vectorstore.as_retriever() # fetches answers when qts asked

In [ ]:
# Define prompt template
prompt = ChatPromptTemplate(
    input_variables = ['context', 'question'],
    messages = [
        HumanMessagePromptTemplate(
            prompt = PromptTemplate(
                input_varibales = ['context', 'question'],
                template = (
                    "You are an assisstant for question-answering tasks."
                    "Use the following retrieved context to answer the question."
                    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
                    "Use three sentences maximum and keep the answer concise."
                    "Question: {question}\n"
                    "Context: {context}\n"
                    "Answer:"
                )
            )
        )
    ]
)

In [ ]:
# Function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Define the RAG chain - its sequence imp
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
# Invoke the RAG chain
response = rag_chain.invoke("What is this article about?")
print(response)

This article explains the difference between descriptive and inferential statistics.  

Descriptive statistics summarize and describe features of a dataset, while inferential statistics use sample data to make predictions about a larger population.  Examples are provided for both types of statistics. 



In [ ]:
response = rag_chain.invoke("What is inferential statistics?")
print(response)

Inferential statistics uses a sample of data to make predictions or inferences about a larger population.  It often involves hypothesis testing or confidence intervals.  For example, inferential statistics could be used to estimate the preferences of an entire city based on a survey of 100 people. 

